# Function Design Guideline
- Use arguments for inputs and  return for outputs
    - make a function independent of things outside of it
- Avoid using global variable and mutable arguments
- Each function should have a single, unified purpose
- Each function should be small
- **Minimize external dependencies in functions and other program components** -> self-containied


---

# Recursive Functions
Recursion is not used as often in Python as in language like Prolog or Lisp, because Python emphasizes simpler procedural statements like loops.  
However, this technique can still be usefull.  

- The misuse of the following operator overloadings might cause inproper recursive behavior
    - `__setattr__`, `__getattribute__`, `__repr__`

Standard Python limits the depth of its runtime call stack to trap infinite recursion.

In [1]:
# To see the recursion limit
import sys
sys.getrecursionlimit()

1000

In [2]:
# To set the recursion limit
sys.setrecursionlimit(10000)
sys.getrecursionlimit()

10000

---

# Function Objects

**First-Class Object model**: Functions are just a kind of object. They can be assigned to other names, passed as arguement, embedded in data strcutures, returned and other operations that other object can do.  

This first-class object model and lacking of type declarations make Python incredibly flexible.

## Function Attributes
It's is possible to attach user-defined attributes to functions

In [4]:
def func():
    print('hi')

func.hi = 'hi'
print(func.hi)

hi


In [5]:
dir(func)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'hi']

Such attributes can be used to attach **state information** to function object.  
This way can be used to emulate "static locals" in other languages - variables whose names are local to a function

## Function Annotations (Python3)